# 01: Global Fits

In [ ]:
import SMEFT19
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from flavio.statistics.functions import pull
import warnings

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    SMEFT19.SMEFTglob.gl.make_measurement()

In [ ]:
def fit_I(x):
    return -SMEFT19.likelihood_global(x, SMEFT19.scenarios.rotBI)

In [ ]:
bf, v, d, L = SMEFT19.ellipse.minimum(fit_I, [-0.13, 0, 0.851])

In [ ]:
SMEFT19.__path__

In [ ]:
SMEFT19.SMEFTglob.likelihood_fits([-0.13, 0, 0.851], SMEFT19.scenarios.rotBI)

In [ ]:
d1 = {5: 'a', 7: 'b', 4.3: 'c'}

In [ ]:
d1[4.3]

In [ ]:
SMEFT19.scenarios.rotBI([-0.13, 0, 0.851]).wc.values.keys()

In [ ]:
from wilson import Wilson

In [ ]:
from collections import defaultdict

In [ ]:
d = defaultdict(float)

In [ ]:
d['a'] += 7

In [ ]:
d['a']

In [ ]:
d['b'] -= 2.3

In [ ]:
d['b']

In [ ]:
dict(d)

In [ ]:
cache_running = {125.0: {}, 91.1876: {}, 4.8: {}, 1.8: {} }

In [ ]:
fit_scale = {'fast_likelihood_quarks.yaml': 4.8,
                 'fast_likelihood_leptons.yaml': 1.8,
                 'likelihood_ewpt.yaml': 91.1876,
                 'likelihood_eeww.yaml': 91.1876,
                 'likelihood_lept.yaml': 1.8,
                 'likelihood_rd_rds.yaml': 4.8,
                 'likelihood_lfu_fccc.yaml': 4.8,
                 'likelihood_lfu_fcnc.yaml': 4.8,
                 'likelihood_bcpv.yaml': 4.8,
                 'likelihood_bqnunu.yaml': 4.8,
                 'likelihood_lfv.yaml': 4.8,
                 'likelihood_zlfv.yaml': 91.1876,
                 'likelihood_higgs.yaml': 125.0
                 }

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    glfit = {}
    for fit in fit_scale.keys():
            scale = fit_scale[fit]
            if scale > 80:
                eft = 'SMEFT'
                basis = 'Warsaw'
            else:
                eft = 'WET'
                basis = 'flavio'
            glfit[fit] = smelli.GlobalLikelihood(eft=eft, include_likelihoods=[fit,])
            glfit[fit].make_measurement()

In [ ]:
import smelli

In [ ]:
def likelihood_fits_cached(x, wfun):
    global cache_running
    res = {}
    for fit in fit_scale.keys():
        scale = fit_scale[fit]
        if scale > 80:
            eft = 'SMEFT'
            basis = 'Warsaw'
        else:
            eft = 'WET'
            basis = 'flavio'
        w0 = wfun(x)
        wc_res = defaultdict(float)
        for wc in w0.wc.values.keys():
            if wc not in cache_running[scale].keys():
                cache_running[scale][wc] = Wilson({wc: 1e-6}, scale=1e3, basis='Warsaw', eft='SMEFT').match_run(scale=scale, eft=eft, basis=basis).values
            for w in cache_running[scale][wc].keys():
                if isinstance(cache_running[scale][wc][w], float):
                    wc_res[w] += cache_running[scale][wc][w]/1e-6 * w0.wc.values[wc]
                else:
                    wc_res[w] += (cache_running[scale][wc][w]['Re']+cache_running[scale][wc][w]['Im'])/1e-6 * w0.wc.values[wc]
        res[fit] = glfit[fit].parameter_point(Wilson(dict(wc_res), scale=scale, eft=eft, basis=basis )).log_likelihood_global()
    res['global'] = sum(res[fit] for fit in fit_scale.keys())
    return res

In [ ]:
t0 = time.time()
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    l1 = likelihood_fits_cached([-0.13, 0, 0.851], SMEFT19.scenarios.rotBI)
tf = time.time()

In [ ]:
import time

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(l1['fast_likelihood_quarks.yaml'].log_likelihood_global())

In [ ]:
tf-t0

In [ ]:
l1['global']

In [ ]:
def fit_I(x):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        return -likelihood_fits_cached(x, SMEFT19.scenarios.rotBI)['global']

In [ ]:
bf_I, v_I, d_I, L_I = SMEFT19.ellipse.minimum(fit_I, [-0.13, 0, 0.851])

In [ ]:
cache_running